In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from transformers import TFBertModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import TFAutoModel, AutoTokenizer, BertForPreTraining
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import TFBertModel
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay)
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

C:\Users\zoika\anaconda3\envs\tf_gpu_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def metrics(y_pred, y_true):
  print("f1 score: ", f1_score(y_true, y_pred))
  print("accuracy: ", accuracy_score(y_true, y_pred))
  print("precision:", precision_score(y_true, y_pred, zero_division=1))
  print("recall: ", recall_score(y_true, y_pred, zero_division=1))

In [3]:
model = TFAutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

path_train = "../../dataset/train_bert_new_4.csv"
path_test = "../../dataset/test_bert_new_4.csv"

df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

x_train = df_train['text'].tolist()
x_test = df_test['text'].tolist()
y_train = df_train['prediction'].tolist()
y_test = df_test['prediction'].tolist()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
def tokenize(sentences):
    return tokenizer(sentences, padding=True, truncation=True, return_tensors="tf")

In [5]:
x_train_tokenized = tokenize(x_train)
x_test_tokenized = tokenize(x_test)

In [7]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes, linear_units=512):
        super().__init__()
        self.bert = bert_model

        self.bert.trainable = False

        self.linear = tf.keras.layers.Dense(linear_units, activation='relu', trainable=True)
        self.dense = tf.keras.layers.Dense(num_classes, activation='softmax', trainable=True)

    def call(self, inputs):
        outputs = self.bert(inputs)
        pooled_output = outputs.pooler_output
        linear_output = self.linear(pooled_output) 
        logits = self.dense(linear_output)

        return logits

num_classes = 2

classifier = BERTForClassification(model, num_classes)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(x_train_tokenized), y_train)).shuffle(100, seed=42).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(x_test_tokenized), y_test)).batch(16)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = classifier.fit(
    train_dataset,
    epochs=100
)


Epoch 1/100
7/7 [==============================] - 16s 372ms/step - loss: 0.7385 - accuracy: 0.5000
Epoch 2/100
7/7 [==============================] - 3s 371ms/step - loss: 0.6580 - accuracy: 0.6204
Epoch 3/100
7/7 [==============================] - 3s 373ms/step - loss: 0.6805 - accuracy: 0.5556
Epoch 4/100
7/7 [==============================] - 3s 376ms/step - loss: 0.6655 - accuracy: 0.6574
Epoch 5/100
7/7 [==============================] - 3s 373ms/step - loss: 0.6592 - accuracy: 0.6481
Epoch 6/100
7/7 [==============================] - 3s 376ms/step - loss: 0.6651 - accuracy: 0.5833
Epoch 7/100
7/7 [==============================] - 3s 376ms/step - loss: 0.6495 - accuracy: 0.6944
Epoch 8/100
7/7 [==============================] - 3s 373ms/step - loss: 0.6342 - accuracy: 0.7130
Epoch 9/100
7/7 [==============================] - 3s 375ms/step - loss: 0.6251 - accuracy: 0.7315
Epoch 10/100
7/7 [==============================] - 3s 375ms/step - loss: 0.6267 - accuracy: 0.7500
Epoch 11

In [8]:
classifier.evaluate(train_dataset)
classifier.evaluate(test_dataset)

3/3 [==============================] - 4s 467ms/step - loss: 0.5073 - accuracy: 0.7292


[0.5072969794273376, 0.7291666865348816]

In [9]:
predictions = classifier.predict(test_dataset)


threshold = 0.5
y_pred_probs = tf.nn.softmax(predictions)  
y_pred_class = tf.argmax(predictions, axis=1)  
# y_pred_thresholded = tf.where(y_pred_probs[:, 1] > threshold, 1, y_pred_class)

accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred_class, y_test), dtype=tf.float32))
print("Accuracy:", accuracy.numpy())

3/3 [==============================] - 4s 461ms/step
Accuracy: 0.7291667


In [10]:
results = metrics(y_test, y_pred_class)

f1 score:  0.6829268292682927
accuracy:  0.7291666666666666
precision: 0.5833333333333334
recall:  0.8235294117647058


In [11]:
import tensorflow as tf

class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes, linear_units=512, additional_linear_units=256, dropout_rate=0.1, trainable_layers=None):
        super().__init__()
        self.bert = bert_model
        self.trainable_layers = trainable_layers


        self.bert.trainable = False


        if trainable_layers is not None:
            for layer in self.bert.layers[-trainable_layers:]:
                layer.trainable = True

        self.linear1 = tf.keras.layers.Dense(linear_units, activation='relu')
        self.linear2 = tf.keras.layers.Dense(additional_linear_units, activation='relu') 
        self.multihead_attention = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=64, dropout=dropout_rate)
        self.dense = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        outputs = self.bert(inputs, training=training)  
        pooled_output = outputs.pooler_output
        linear_output1 = self.linear1(pooled_output)  
        linear_output2 = self.linear2(linear_output1) 
        linear_output2 = tf.expand_dims(linear_output2, axis=1) 
        attention_output = self.multihead_attention(linear_output2, linear_output2) 
        attention_output = tf.squeeze(attention_output, axis=1) 
        logits = self.dense(attention_output)

        return logits


num_classes = 2

trainable_layers = 10

classifier = BERTForClassification(model, num_classes, trainable_layers=trainable_layers)

train_dataset = tf.data.Dataset.from_tensor_slices((dict(x_train_tokenized), y_train)).shuffle(100, seed = 42).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(x_test_tokenized), y_test)).batch(16)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = classifier.fit(
    train_dataset,
    epochs=73
)


Epoch 1/73
7/7 [==============================] - 12s 408ms/step - loss: 0.6924 - accuracy: 0.5000
Epoch 2/73
7/7 [==============================] - 3s 423ms/step - loss: 0.6909 - accuracy: 0.5185
Epoch 3/73
7/7 [==============================] - 3s 422ms/step - loss: 0.6889 - accuracy: 0.5185
Epoch 4/73
7/7 [==============================] - 3s 426ms/step - loss: 0.6837 - accuracy: 0.5556
Epoch 5/73
7/7 [==============================] - 3s 415ms/step - loss: 0.6874 - accuracy: 0.5463
Epoch 6/73
7/7 [==============================] - 3s 421ms/step - loss: 0.6837 - accuracy: 0.6111
Epoch 7/73
7/7 [==============================] - 3s 412ms/step - loss: 0.6782 - accuracy: 0.6759
Epoch 8/73
7/7 [==============================] - 3s 422ms/step - loss: 0.6758 - accuracy: 0.6111
Epoch 9/73
7/7 [==============================] - 3s 424ms/step - loss: 0.6733 - accuracy: 0.6574
Epoch 10/73
7/7 [==============================] - 3s 430ms/step - loss: 0.6712 - accuracy: 0.6111
Epoch 11/73
7/7 [=

In [12]:
predictions = classifier.predict(test_dataset)


threshold = 0.5
y_pred_probs = tf.nn.softmax(predictions)  
y_pred_class = tf.argmax(predictions, axis=1)  
# y_pred_thresholded = tf.where(y_pred_probs[:, 1] > threshold, 1, y_pred_class)

accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred_class, y_test), dtype=tf.float32))
print("Accuracy:", accuracy.numpy())

3/3 [==============================] - 4s 456ms/step
Accuracy: 0.75


In [13]:
metrics(y_test, y_pred_class)

f1 score:  0.6666666666666666
accuracy:  0.75
precision: 0.5
recall:  1.0
